# DATA

In [285]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [286]:
vendas = pd.read_csv('../../../data/interim/Vendas.csv', encoding='ISO-8859-1')

In [287]:
vendas = vendas.replace(['ST', 'CORSÁRIO SUPLEX', 'ESP', 'ESPECIAL'], ['00','CORSARIO SUPLEX', 'GG', 'GG'])
try:
    df_metrics.empty
except:
    df_metrics = pd.DataFrame(columns=['XGB', 'GBR'])
vendas.head()

,ANO,DESCRICAO,DIA,HORA,MES,NOME,QUANTIDADE,TAMANHO,VLR_UNIT
0,2016.0,CAMISETA MANGA CURTA,12.0,14:54,1.0,VERBO DIVINO,7.0,00,35.5
1,2016.0,BERMUDA CICLISTA COTTON,12.0,15:08,1.0,VERBO DIVINO,1.0,08,51.0
2,2016.0,CAMISETA MANGA CURTA,12.0,15:08,1.0,VERBO DIVINO,5.0,PP,37.5
3,2016.0,BLUSA MOLETOM FLANELADA,12.0,15:27,1.0,VERBO DIVINO,1.0,M,115.0
4,2016.0,CALÇA SARJA MASC,12.0,15:27,1.0,VERBO DIVINO,1.0,GG,116.0


In [288]:
df_metrics

,GBR,XGB
0,10.790752,10.810558
1,6.508859,6.582399
2,5.735807,5.858361
3,5.587040,5.559623
4,5.571515,5.584030


In [289]:
a = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO'])

In [290]:
df = a.sum().reset_index().drop(['DIA', 'VLR_UNIT'], 1)
df = df.sort_values(by=['ANO','MES', 'DESCRICAO', 'TAMANHO', 'NOME'])

In [291]:
df_vlr_unit = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO']).mean()
df = df.merge(df_vlr_unit.drop(['QUANTIDADE', 'DIA'], axis=1), on=['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO'], how='left')

In [292]:
cols = df.columns.tolist()
y = cols.index('QUANTIDADE')
cols = cols[:y] + cols[y+1:] + ['QUANTIDADE']
df = df[cols]

In [293]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,M,57.5,1.0
1,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,P,57.5,1.0
2,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,P,57.5,1.0
3,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,PP,57.5,1.0
4,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ITAIM,PP,57.5,1.0


In [294]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,M,57.5,1.0
1,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,P,57.5,1.0
2,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,P,57.5,1.0
3,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,PP,57.5,1.0
4,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ITAIM,PP,57.5,1.0


In [295]:
treino.shape, validacao.shape

((4396, 11), (985, 11))

In [296]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [297]:
le_DESC = LabelEncoder()
# treino['DESCRICAO'] = le_DESC.fit_transform(treino.DESCRICAO)
# validacao['DESCRICAO'] = le_DESC.transform(validacao.DESCRICAO)
df['DESCRICAO'] = le_DESC.fit_transform(df.DESCRICAO)

In [298]:
tamanho_val = {
    '00' : 0,
    '02' : 1,
    '04' : 2,
    '06' : 3,
    '08' : 4,
    '10' : 5,
    '12' : 6,
    '14' : 7,
    'PP' : 8,
    'P' : 9,
    'M' : 10,
    'G' : 11,
    'GG' : 12
}
# treino.TAMANHO = treino.TAMANHO.apply(lambda x: tamanho_val[x])
# validacao.TAMANHO = validacao.TAMANHO.apply(lambda x: tamanho_val[x])
df.TAMANHO = df.TAMANHO.apply(lambda x: tamanho_val[x])

In [299]:
le_NOME = LabelEncoder()
# treino['NOME'] = le_NOME.fit_transform(treino.NOME)
# validacao['NOME'] = le_NOME.transform(validacao.NOME)
df['NOME'] = le_NOME.fit_transform(df.NOME)

In [300]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,0,3,10,57.5,1.0
1,2016.0,1.0,0,1,9,57.5,1.0
2,2016.0,1.0,0,3,9,57.5,1.0
3,2016.0,1.0,0,1,8,57.5,1.0
4,2016.0,1.0,0,2,8,57.5,1.0


# FEATURES

In [301]:
# descricao - mes
desc_mes = df.groupby(['DESCRICAO', 'MES']).QUANTIDADE.sum().reset_index(name='DESCRICAO_MES')
df = df.merge(desc_mes, on=['DESCRICAO', 'MES'], how='left')
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE,DESCRICAO_MES
0,2016.0,1.0,0,3,10,57.5,1.0,16.0
1,2016.0,1.0,0,1,9,57.5,1.0,16.0
2,2016.0,1.0,0,3,9,57.5,1.0,16.0
3,2016.0,1.0,0,1,8,57.5,1.0,16.0
4,2016.0,1.0,0,2,8,57.5,1.0,16.0


In [302]:
desc_place = df.groupby(['DESCRICAO', 'NOME']).QUANTIDADE.sum().reset_index(name='DESCRICAO_NOME')
df = df.merge(desc_place, on=['DESCRICAO', 'NOME'], how='left')
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE,DESCRICAO_MES,DESCRICAO_NOME
0,2016.0,1.0,0,3,10,57.5,1.0,16.0,14.0
1,2016.0,1.0,0,1,9,57.5,1.0,16.0,3.0
2,2016.0,1.0,0,3,9,57.5,1.0,16.0,14.0
3,2016.0,1.0,0,1,8,57.5,1.0,16.0,3.0
4,2016.0,1.0,0,2,8,57.5,1.0,16.0,4.0


In [303]:
# year_month = df[['ANO', 'MES', 'QUANTIDADE', 'DESCRICAO', 'TAMANHO']].groupby(['ANO', 'MES', 'DESCRICAO', 'TAMANHO']).sum().reset_index()
# def get_last_year_sells(data):

#     qnt = year_month[(year_month.ANO == data['ANO'] - 1) & (year_month.MES == data['MES'])]
#     if qnt.empty:
#         qnt = year_month[(year_month.ANO == data['ANO']) & 
#                          (year_month.MES == data['MES']) & 
#                          (year_month.DESCRICAO == data['DESCRICAO']) &
#                          (year_month.TAMANHO == data['TAMANHO'])].QUANTIDADE.values
#         if qnt.size > 0:
#             return qnt[0]
#         else:
#             return 0
#     else:
#         qnt = qnt[(qnt.DESCRICAO == data['DESCRICAO']) & (qnt.TAMANHO == data['TAMANHO'])].QUANTIDADE.values
#         if qnt.size > 0:
#             return qnt[0]
#         else:
#             return 0
# df['last_year_sell'] = df.apply(get_last_year_sells, axis=1)

In [304]:
# def clima(mes):
#     if mes in [1, 2, 3]:
#         return 1 #verão
#     elif mes in [4,5,6]:
#         return 2 #outono
#     elif mes in [7,8,9]:
#         return 3 #inverno
#     else:
#         return 4#primavera
# df['ESTACAO'] = df.MES.apply(clima)
# df.ESTACAO = df.ESTACAO.apply(lambda x: x > 1 and x < 4)
# est_desc = df.groupby(['ESTACAO', 'DESCRICAO']).QUANTIDADE.sum().reset_index(name='DESCRICAO_ESTACAO')
# df = df.merge(est_desc, on=['ESTACAO', 'DESCRICAO'], how='left')
# df.head()

# TRAIN

In [306]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE,DESCRICAO_MES,DESCRICAO_NOME
0,2016.0,1.0,0,3,10,57.5,1.0,16.0,14.0
1,2016.0,1.0,0,1,9,57.5,1.0,16.0,3.0
2,2016.0,1.0,0,3,9,57.5,1.0,16.0,14.0
3,2016.0,1.0,0,1,8,57.5,1.0,16.0,3.0
4,2016.0,1.0,0,2,8,57.5,1.0,16.0,4.0


In [317]:
treino = df[(df.ANO < 2018)].drop(['ANO'], axis=1)
validacao = df[df.ANO >= 2018].drop(['ANO'], axis=1)
Y_train = treino['QUANTIDADE']
X_train = treino.drop('QUANTIDADE', 1)
X_test = validacao.drop('QUANTIDADE', 1)
Y_test = validacao['QUANTIDADE']
scaler = StandardScaler().fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

In [318]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBRegressor
from math import sqrt

In [313]:
models = []
models.append(('GBR', GradientBoostingRegressor()))
models.append(('XGB', XGBRegressor()))
# models.append(('XGB_grid', grid.best_estimator_))
results = []
new_metrics = {}
print(X_train.columns)
for name, model in models:
    if name in ['LR', 'LDA']:
        X_train_case = X_train_sc
        X_test_case = X_test_sc
    else:
        X_train_case = X_train
        X_test_case = X_test
    model.fit(X_train_case, Y_train)
    Y_pred = model.predict(X_test_case)
    metrics = sqrt(mean_squared_error(Y_test, Y_pred))
    new_metrics[name] = metrics
    print(name, metrics)
    print(model.feature_importances_)
if df_metrics.empty:
    df_metrics = df_metrics.append(new_metrics, ignore_index=True)
else:
    min_GBR = df_metrics.GBR.min()
    min_XGB = df_metrics.values.min()
#     min_grid = df_metrics.XGB_grid.min()
    if (new_metrics['GBR'] < min_GBR and new_metrics['GBR'] < min_XGB) or (new_metrics['XGB'] < min_XGB and new_metrics['XGB'] < min_GBR):
        df_new_metric = pd.DataFrame(new_metrics, index=[0])
        df_metrics = df_metrics.append(df_new_metric, ignore_index=True)

Index(['MES', 'DESCRICAO', 'NOME', 'TAMANHO', 'VLR_UNIT', 'DESCRICAO_MES',
       'DESCRICAO_NOME'],
      dtype='object')
GBR 5.534733301219784
[0.07357587 0.007832   0.20130382 0.21721008 0.04881858 0.31324775
 0.1380119 ]
XGB 5.57695270333956
[0.09142857 0.02714286 0.20142858 0.21428572 0.08285714 0.25714287
 0.12571429]


In [314]:
df_metrics

,GBR,XGB
0,10.790752,10.810558
1,6.508859,6.582399
2,5.735807,5.858361
3,5.587040,5.559623
4,5.571515,5.584030
5,5.545486,5.585083
6,5.534733,5.576953


In [320]:
models = []
models.append(('GBR', GradientBoostingRegressor()))
models.append(('XGB', XGBRegressor()))
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
for name, model in models:
    metrics = []
    for year in df.ANO.unique()[1:]:
        treino = df[df.ANO == year -1].drop('ANO', axis=1)
        validacao = df[df.ANO == year].drop('ANO', axis=1)
        Y_train = treino['QUANTIDADE']
        X_train = treino.drop('QUANTIDADE', 1)
        X_test = validacao.drop('QUANTIDADE', 1)
        Y_test = validacao['QUANTIDADE']
        if name in ['LR', 'LDA']:
            scaler = StandardScaler().fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        metrics.append(sqrt(mean_squared_error(Y_test, Y_pred)))
    print(name, np.mean(metrics))

GBR 6.499681805671081
XGB 6.368585666704458
LR 12.942419058548735
LDA 14.61361438243139


# XGBOOST GRID SEARCH

In [86]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [89]:
XGB = XGBRegressor(learning_rate=.02, n_estimators=600, nthread=1)

In [96]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, GridSearchCV

In [95]:
folds = 5
param_comb = 20
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(
    XGB, param_distributions=params, n_iter=param_comb, scoring='neg_mean_squared_error',
    n_jobs=4, cv=skf.split(X_train,Y_train), verbose=3, random_state=1001 )

In [97]:
random_search.fit(X_train, Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/chazanas/.virtualenvs/sold_alt/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=-32.904173404263695, total=   0.7s
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=-300.5451710071674, total=   0.7s
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=0.6, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=0.8 
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=-25.201480612471226, total=   0.6s
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=-26.259217432254218, total

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.5s


[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6, score=-27.378083579824967, total=   0.7s
[CV] subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6 
[CV]  subsample=0.8, min_child_weight=1, max_depth=4, gamma=1, colsample_bytree=1.0, score=-32.66564149850497, total=   1.1s
[CV] subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6, score=-271.051621442313, total=   0.7s
[CV] subsample=1.0, min_child_weight=1, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6, score=-26.14852362738454, total=   0.8s
[CV] subsample=1.0, min_child_weight=1, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6, score=-23.21251614305254, total=   0.7s
[CV] subsample=1.0, m

[CV]  subsample=0.6, min_child_weight=10, max_depth=5, gamma=1, colsample_bytree=0.6, score=-21.031789511953733, total=   1.3s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=0.6, min_child_weight=10, max_depth=5, gamma=1, colsample_bytree=0.6, score=-27.78262004111416, total=   1.2s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=-24.21904912922863, total=   1.4s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=-187.72210253650698, total=   1.4s
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=-21.82489947365507, total=   1.3s
[CV] subsample=1.0, m

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   30.9s finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x10ff13200>,
          error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=20, n_jobs=4,
          param_distributions={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=3)

In [108]:
grid = GridSearchCV(estimator=XGB, param_grid=params, scoring='neg_mean_squared_error', n_jobs=4, cv=skf.split(X_train,Y_train), verbose=3 )
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 405 candidates, totalling 2025 fits
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6, score=-293.6511575702421, total=   0.6s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6, score=-24.93049218349676, total=   0.6s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6, score=-25.104298835327977, total=   0.6s
[CV] colsample_bytree=0.6, gamma=0.5, 

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    4.8s


[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=1.0, score=-29.3708541698202, total=   0.6s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=1.0 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=1.0, score=-302.2045650717183, total=   0.6s
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=0.8, score=-38.8502821280852, total=   0.7s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=1.0 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=10, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=1.0, score=-24.734695628862813, total=   0.6s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=10, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=1.0, score=-24.668513208692875, total=   0.6s
[CV] colsample_bytr

[CV] colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.6, score=-22.940716483912855, total=   0.8s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.6, score=-32.13444420014682, total=   0.9s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.8, score=-29.503510229322135, total=   0.8s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.8, score=-266.190734764528, total=   0.8s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=1.0 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=4, min_child

[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=1, subsample=1.0, score=-245.7680973848381, total=   1.1s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=1, subsample=1.0, score=-23.651427030796427, total=   1.1s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=1, subsample=1.0, score=-20.780255795664292, total=   1.0s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=1, subsample=1.0, score=-28.191500527865802, total=   1.0s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=5, subsample=0.6, score=-28.303600197689114, total=   1.4s
[CV] colsample_by

...

[CV] colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=0.8 
[CV] colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=0.8, score=-25.82945696172238, total=   0.8s
[CV] colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=0.8, score=-24.4790547743852, total=   0.8s
[CV] colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0, score=-32.904173404263695, total=   0.7s
[CV] colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=0.8, score=-37.518244729346726, total=   0.8s
[CV] colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=

[CV] colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.6, score=-227.29675220051917, total=   1.2s
[CV] colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.6, score=-25.57347905572776, total=   1.2s
[CV] colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.6, score=-20.45924621511838, total=   1.1s
[CV] colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.6, score=-35.183645075833454, total=   1.1s
[CV] colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=4, min_child_weight=5, subsampl

[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, subsample=0.8, score=-30.34142879030393, total=   1.5s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, subsample=1.0 
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, subsample=1.0, score=-189.86925953274658, total=   1.4s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, subsample=1.0, score=-21.440832322914645, total=   1.3s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, subsample=1.0, score=-30.920832246940208, total=   1.2s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, subsamp

[Parallel(n_jobs=4)]: Done 2025 out of 2025 | elapsed:  9.3min finished


GridSearchCV(cv=<generator object _BaseKFold.split at 0x11068b200>,
       error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=3)

In [128]:
XGB = grid.best_estimator_
XGB.fit(X_train, Y_train)
Y_pred = XGB.predict(X_test)
sqrt(mean_squared_error(Y_test, Y_pred))

6.105038772594109

# TESTES

In [28]:
vendas.QUANTIDADE.unique()

array([ 7.,  1.,  5.,  6.,  2.,  3.,  4., 12.,  8.,  9., 10.])

In [85]:
treino.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,0,3,10,57.5,1.0
1,2016.0,1.0,0,1,9,57.5,1.0
2,2016.0,1.0,0,3,9,57.5,1.0
3,2016.0,1.0,0,1,8,57.5,1.0
4,2016.0,1.0,0,2,8,57.5,1.0


In [171]:
df_metrics.empty

True

In [173]:
df.ANO.min()

2016.0

In [ ]:
x_grid = o